In [1]:
from openai import OpenAI
import backoff
from dotenv import load_dotenv
import os
from fastapi import HTTPException

import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [8]:
name = "asdf Assistant"
instructions = """You're an AI designed to engage in exciting, stimulating, and informative, non-political conversations with your discussion partner. 
                    Your goal is to keep your discussion partner engaged while avoiding any political topic. Make sure to be friendly, introduce yourself, 
                    and to use all your knowledge of non-political subjects to stimulate and engage your discussion partner. For instance you may: 
                    1. Start asking your discussion partner about hobbies and interests and then talk about things they care about, 
                    or propose yourself topics if they do not want to mention personal hobbies or interests; 
                    2. Create interesting and unexpected connections or intriguing facts on varied subjects like science, food, traveling, technology, 
                    health, sports, arts, entertainment...; 
                    3. Let the discussion partner dream suggesting activities or general interest topics to create a spark of enthusiasm. 
                    Important things to consider are: 
                    1. You have to speak German in the conversation, unless prompted otherwise by your discussion partner; 
                    2. You have to make sure not to overwhelm your discussion partner with too many questions or too much information, 
                    limiting yourself to 1 question per message; 
                    3. Avoid all political discussions and maintain a neutral stance on any issue; 
                    4. Use simple and clear language that resonates to the average German person; 
                    5. Keep your messages short and craft each message to be effective but concise, containing 1 to 3 sentences to inform and engage the user.    
                    """    

In [9]:
assistant = client.beta.assistants.create(
            name=name,
            instructions=instructions,
            model="gpt-4o",
            tools=[{"type": "file_search"}],
        )


In [10]:
system_prompt = f"""
    Be friendly, introduce yourself, and to use all your knowledge of non-political subjects to stimulate and engage your discussion partner. 
    Remember to engage and stimulate your conversation partner. Speak in German using a clear language adapted to your discussion partner, 
    gracefully avoid and redirect any political issue, and keep your messages short (1 to 3 sentences).
    """

conversation_start = [{
        "role": "user",
        "content": system_prompt,
    }]

In [11]:
thread = client.beta.threads.create(
                messages=conversation_start,
                )

In [12]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)


In [13]:
run.status

'completed'

In [14]:
response = client.beta.threads.messages.list(
    thread_id=thread.id)


In [15]:
response.data[0].content[0].text.value

'Hallo! Ich bin dein KI-Gesprächspartner für spannende Unterhaltungen. Was sind deine Hobbys und Interessen? Ich freue mich darauf, über alles Mögliche zu plaudern, sei es Reisen, Technologie, Sport oder etwas ganz anderes!'

In [2]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
  
assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-3.5-turbo",
)

In [3]:
thread = client.beta.threads.create()

In [4]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [5]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [6]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_DnW4KcJQpvd4XsEQE5X9jMyW', assistant_id='asst_qw1fRp3lAw7pj4AFzUXAU5jf', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\). Therefore, the value of x that satisfies the equation is 1. \n\nIf you have any more equations you need help with or any other questions, feel free to ask!'), type='text')], created_at=1717743556, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_ECl7EDqeWxVIJqY3WNYITl6q', status=None, thread_id='thread_2sx4vEB5bYsTd8llFiNuM8wu'), Message(id='msg_hzGxIOIDUhJ6iDW4fRIUC9hr', assistant_id='asst_qw1fRp3lAw7pj4AFzUXAU5jf', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Of course, I can help you solve the equation. To solve for x in the equation \\(3x + 11 = 14\\), we need to isolate x on one side of the equatio

In [7]:
run.status

'completed'

In [30]:
import asyncio
import logging
import backoff
from fastapi import HTTPException

logger = logging.getLogger(__name__)

@backoff.on_exception(backoff.expo, Exception, max_tries=5)
async def create_casual_conversation(assistant):
    """
    Takes assistant as input.
    
    Creates a new casual conversation thread.
    
    Returns the created thread and first message.
    """
    try:
        # Create a new conversation thread
        thread = client.beta.threads.create(
            messages=get_casual_conversation(),
        )
        logger.info(f"Casual conversation thread created with ID: {thread.id}")
        
        # Create and poll the run
        run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant)
        logger.debug(f"Casual conversation run created with ID: {run.id}")

        # Polling for the run status to be completed
        max_retries = 10
        retry_count = 0
        while run.status != 'completed' and retry_count < max_retries:
            logger.info(f"Run status: {run.status}, retrying in 2 seconds...")
            await asyncio.sleep(2)
            run = client.beta.threads.runs.get(run.id)
            retry_count += 1

        if run.status == 'completed':
            logger.debug("Run status is completed")
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            logger.debug(f"Messages: {messages}")
            first_message = messages.data[0].content[0].text.value
            logger.debug("First message: %s", first_message)
            return thread, first_message
        else:
            logger.error(f"Run did not complete after {max_retries} retries. Final status: {run.status}")
            raise HTTPException(status_code=500, detail=f"Run did not complete in the expected time frame. Final status: {run.status}")

    except Exception as e:
        logger.error("Failed to create casual conversation:", e)
        raise HTTPException(status_code=500, detail=f"Failed to create casual conversation: {str(e)}")

# Example usage within an async function
async def main():
    assistant_id = "your_assistant_id"
    try:
        thread, first_message = await create_casual_conversation(assistant_id)
        print("Thread ID:", thread.id)
        print("First Message:", first_message)
    except HTTPException as e:
        print(e.detail)

# Run the main function
await main()


--- Logging error ---
Traceback (most recent call last):
  File "/tmp/ipykernel_4887/2743679753.py", line 20, in create_casual_conversation
    messages=get_casual_conversation(),
             ^^^^^^^^^^^^^^^^^^^^^^^
NameError: name 'get_casual_conversation' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/mo/.pyenv/versions/3.11.8/lib/python3.11/logging/__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/mo/.pyenv/versions/3.11.8/lib/python3.11/logging/__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/home/mo/.pyenv/versions/3.11.8/lib/python3.11/logging/__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/home/mo/.pyenv/versions/3.11.8/lib/python3.11/logging/__init__.py", line 377, in getMessage
    msg = msg % self.args
  

Failed to create casual conversation: name 'get_casual_conversation' is not defined


--- Logging error ---
Traceback (most recent call last):
  File "/tmp/ipykernel_4887/2743679753.py", line 20, in create_casual_conversation
    messages=get_casual_conversation(),
             ^^^^^^^^^^^^^^^^^^^^^^^
NameError: name 'get_casual_conversation' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/mo/.pyenv/versions/3.11.8/lib/python3.11/logging/__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/mo/.pyenv/versions/3.11.8/lib/python3.11/logging/__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/home/mo/.pyenv/versions/3.11.8/lib/python3.11/logging/__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/home/mo/.pyenv/versions/3.11.8/lib/python3.11/logging/__init__.py", line 377, in getMessage
    msg = msg % self.args
  

In [28]:
!pip install nest_asyncio
